In [203]:
import numpy as np
import pandas as pd
import yfinance as yf
import warnings
warnings.filterwarnings('ignore')

In [204]:
tickers=['ONGC.NS','TCS.NS','PATANJALI.NS','RELIANCE.NS','SBIN.NS']
data=yf.download(tickers,start='2019-01-01',end='2023-01-01')['Close']
market=yf.download('^GSPC',start='2019-01-01',end='2023-01-01')['Close']
data=data.dropna()
market=market.dropna()

[*********************100%%**********************]  5 of 5 completed
[*********************100%%**********************]  1 of 1 completed


In [205]:
aligned_data = data.join(market, how='inner', rsuffix='_Market').dropna()

In [206]:
data

Ticker,ONGC.NS,PATANJALI.NS,RELIANCE.NS,SBIN.NS,TCS.NS
Date,,,,,
2019-01-01,148.500000,855.000000,1024.966919,299.600006,1902.800049
2019-01-02,147.050003,815.000000,1011.617737,293.899994,1923.300049
2019-01-03,142.399994,775.000000,999.137085,291.100006,1899.949951
2019-01-04,145.699997,810.000000,1004.531616,297.649994,1876.849976
2019-01-07,147.850006,850.000000,1010.109070,296.299988,1897.900024
...,...,...,...,...,...
2022-12-26,141.300003,1145.199951,2329.697266,597.099976,3252.899902
2022-12-27,144.800003,1151.250000,2348.757324,601.900024,3259.500000
2022-12-28,143.899994,1160.949951,2348.526611,601.049988,3257.100098


In [207]:
aligned_data

,ONGC.NS,PATANJALI.NS,RELIANCE.NS,SBIN.NS,TCS.NS,Close
Date,,,,,,
2019-01-02,147.050003,815.000000,1011.617737,293.899994,1923.300049,2510.030029
2019-01-03,142.399994,775.000000,999.137085,291.100006,1899.949951,2447.889893
2019-01-04,145.699997,810.000000,1004.531616,297.649994,1876.849976,2531.939941
2019-01-07,147.850006,850.000000,1010.109070,296.299988,1897.900024,2549.689941
2019-01-08,147.449997,815.000000,1010.017639,305.299988,1893.550049,2574.409912
...,...,...,...,...,...,...
2022-12-23,139.800003,1101.150024,2309.529785,574.000000,3228.350098,3844.820068
2022-12-27,144.800003,1151.250000,2348.757324,601.900024,3259.500000,3829.250000
2022-12-28,143.899994,1160.949951,2348.526611,601.049988,3257.100098,3783.219971


In [208]:
stocks_return=aligned_data.pct_change()

In [209]:
market_return=stocks_return['Close'].mean()*252

In [210]:
market_cap={
    'ONGC.NS':yf.Ticker('ONGC.NS').info['marketCap'],
    'TCS.NS':yf.Ticker('TCS.NS').info['marketCap'],
    'PATANJALI.NS':yf.Ticker('PATANJALI.NS').info['marketCap'],
    'RELIANCE.NS':yf.Ticker('RELIANCE.NS').info['marketCap'],
    'SBIN.NS':yf.Ticker('SBIN.NS').info['marketCap']
   }


In [211]:
total_market_cap=sum(market_cap.values())

In [212]:
key=list(market_cap.values())

In [213]:
weights={
    'weights':[key[0]/total_market_cap,key[1]/total_market_cap,key[2]/total_market_cap,key[3]/total_market_cap,key[4]/total_market_cap]
}


In [214]:
weights=pd.DataFrame(weights)
weights.index=tickers

In [215]:
weights

,weights
ONGC.NS,0.076396
TCS.NS,0.305721
PATANJALI.NS,0.011842
RELIANCE.NS,0.440894
SBIN.NS,0.165147


In [216]:
def Beta(data,marke_return):
    numerator=np.cov(data,marke_return)
    numerator=numerator[0][1]
    denominator=np.var(marke_return)
    beta=numerator/denominator
    return beta

In [217]:
beta={
    'Beta':[Beta(aligned_data['ONGC.NS'],aligned_data['Close']),Beta(aligned_data['TCS.NS'],aligned_data['Close']),Beta(aligned_data['PATANJALI.NS'],aligned_data['Close']),Beta(aligned_data['RELIANCE.NS'],aligned_data['Close']),Beta(aligned_data['SBIN.NS'],aligned_data['Close'])]
    }
beta=pd.DataFrame(beta)
beta.index=tickers

In [218]:
beta

,Beta
ONGC.NS,0.013023
TCS.NS,0.986418
PATANJALI.NS,0.314124
RELIANCE.NS,0.648868
SBIN.NS,0.139180


In [219]:
weighted_beta_stocks={
    'weights':[beta['Beta'].iloc[0]*weights['weights'].iloc[0],beta['Beta'].iloc[1]*weights['weights'].iloc[1],beta['Beta'].iloc[2]*weights['weights'].iloc[2],beta['Beta'].iloc[3]*weights['weights'].iloc[3],beta['Beta'].iloc[4]*weights['weights'].iloc[4]]
}
weighted_beta_stocks=pd.DataFrame(weighted_beta_stocks)
weighted_beta_stocks.index=tickers

In [220]:
weighted_beta_stocks

,weights
ONGC.NS,0.000995
TCS.NS,0.301569
PATANJALI.NS,0.003720
RELIANCE.NS,0.286082
SBIN.NS,0.022985


In [221]:
def expected_return(beta,market_return,risk_free_rate=0.02):
    return risk_free_rate+beta*(market_return-risk_free_rate)


In [222]:
returns=[]
for i in range(len(beta)):
    returns.append(expected_return(beta['Beta'].iloc[i],market_return))

In [223]:
risk_free_rate=0.02
df={
    'Stock Return':returns
}
df=pd.DataFrame(df)
df.index=tickers

In [224]:
df

,Stock Return
ONGC.NS,0.021541
TCS.NS,0.136696
PATANJALI.NS,0.057162
RELIANCE.NS,0.096763
SBIN.NS,0.036465


In [225]:
portfolio_return=0
for i in range(len(beta)):
    portfolio_return+=df['Stock Return'].iloc[i]*weights['weights'].iloc[i]

Percentage Portfolio Return

In [226]:
portfolio_return*100

9.279763803337474

ValueError: shapes (6,6) and (5,) not aligned: 6 (dim 1) != 5 (dim 0)

array([0.02154063, 0.1366958 , 0.05716164, 0.09676284, 0.03646531])